# XGBoost: HyperOpt of The Parameters


In [1]:
import pandas as pd
import numpy as np

import datetime as dt
from typing import Tuple, List, Dict

from sklearn.metrics import auc, roc_curve


from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, LabelEncoder

from xgboost import XGBClassifier

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from hyperopt.pyll.base import scope

from functools import partial

import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# main flow
start_time = dt.datetime.now()
print("Started at ", start_time)

Started at  2021-03-23 11:13:54.886452


In [3]:
# read data
in_kaggle = False


def get_data_file_path(is_in_kaggle: bool) -> Tuple[str, str, str]:
    train_path = ''
    test_path = ''
    sample_submission_path = ''

    if is_in_kaggle:
        # running in Kaggle, inside the competition
        train_path = '../input/tabular-playground-series-mar-2021/train.csv'
        test_path = '../input/tabular-playground-series-mar-2021/test.csv'
        sample_submission_path = '../input/tabular-playground-series-mar-2021/sample_submission.csv'
    else:
        # running locally
        train_path = 'data/train.csv'
        test_path = 'data/test.csv'
        sample_submission_path = 'data/sample_submission.csv'

    return train_path, test_path, sample_submission_path


In [4]:
%%time
# get the training set and labels
train_set_path, test_set_path, sample_subm_path = get_data_file_path(in_kaggle)

train = pd.read_csv(train_set_path)
test = pd.read_csv(test_set_path)
target = train.target

test_id = test.id

subm = pd.read_csv(sample_subm_path)

Wall time: 1.98 s


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      300000 non-null  int64  
 1   cat0    300000 non-null  object 
 2   cat1    300000 non-null  object 
 3   cat2    300000 non-null  object 
 4   cat3    300000 non-null  object 
 5   cat4    300000 non-null  object 
 6   cat5    300000 non-null  object 
 7   cat6    300000 non-null  object 
 8   cat7    300000 non-null  object 
 9   cat8    300000 non-null  object 
 10  cat9    300000 non-null  object 
 11  cat10   300000 non-null  object 
 12  cat11   300000 non-null  object 
 13  cat12   300000 non-null  object 
 14  cat13   300000 non-null  object 
 15  cat14   300000 non-null  object 
 16  cat15   300000 non-null  object 
 17  cat16   300000 non-null  object 
 18  cat17   300000 non-null  object 
 19  cat18   300000 non-null  object 
 20  cont0   300000 non-null  float64
 21  cont1   30

In [6]:
cols_to_drop = ['id', 'cat5', 'cat7', 'cat8', 'cat10']

cat_cols = [feature for feature in train.columns if 'cat' in feature and feature not in cols_to_drop ]
cont_cols = [feature for feature in train.columns if 'cont' in feature and feature not in cols_to_drop]

all_features = cat_cols + cont_cols

def preprocess(df, encoder=None,
               scaler=None, cols_to_drop=None,
               cols_to_encode=None, cols_to_scale=None):
    """
    Preprocess input data
    :param df: DataFrame with data
    :param encoder: encoder object with fit_transform method
    :param scaler: scaler object with fit_transform method
    :param cols_to_drop: columns to be removed
    :param cols_to_encode: columns to be encoded
    :param cols_to_scale: columns to be scaled
    :return: DataFrame
    """

    if encoder:
        for col in cols_to_encode:
            df[col] = encoder.fit_transform(df[col])

    if scaler:
        for col in cols_to_scale:
            df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

    if cols_to_drop:
        df = df.drop(cols_to_drop, axis=1)

    return df

In [7]:
train = preprocess(train, encoder=LabelEncoder(), scaler=StandardScaler(),
                  cols_to_drop=cols_to_drop, cols_to_encode=cat_cols,
                  cols_to_scale=cont_cols)

# encoder=LabelEncoder()
test = preprocess(test, encoder=LabelEncoder(), scaler=StandardScaler(),
                 cols_to_drop=cols_to_drop, cols_to_encode=cat_cols,
                 cols_to_scale=cont_cols)

In [8]:
# ------------------------------------------------------------------------------
# Parameters
# ------------------------------------------------------------------------------
N_FOLDS = 10
N_ESTIMATORS = 30000
SEED = 2021
BAGGING_SEED = 48

In [9]:
xgb_search_params = {
        "num_leaves": scope.int(hp.quniform("num_leaves", 31, 250, 1)),
        "max_depth": scope.int(hp.quniform("max_depth", 10, 24, 1)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
        'subsample_for_bin': scope.int(hp.quniform('subsample_for_bin', 10000, 300000, 10000)),
        'min_data_in_leaf': scope.int(hp.qloguniform('min_data_in_leaf', 0, 6, 1)),
    
        'min_split_gain': hp.uniform('min_split_gain', 0, 1.0),
        'min_child_samples': scope.int(hp.quniform("min_child_samples", 2, 700, 1)),
        "subsample": hp.uniform("subsample", 0.2, 1.0), #alias: bagging_fraction
        "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1.0), # alias: feature_fraction
        'reg_alpha': hp.uniform('reg_alpha', 1e-5, 1.0),
        'reg_lambda': hp.uniform('reg_lambda', 0, 50),
    
        'min_sum_hessian_in_leaf': None, #overrides min_child_weight
        'min_child_weight': hp.loguniform('min_child_weight', -16, 5), #also aliases to min_sum_hessian
        'min_child_samples': None, #overridden by min_data_in_leaf
        'n_estimators': 2000
}

def xgb_search(params):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

    model = XGBClassifier(**params) 

    evaluation = [(X_test, y_test)]

    model.fit(X_train, y_train,
              eval_set=evaluation, eval_metric='auc',
              early_stopping_rounds=100, verbose=False)

    val_preds = model.predict(X_test)
    
    fpr, tpr, thresholds = roc_curve(y_test, val_preds, pos_label = 1)
    auc_score = auc(fpr, tpr)
    
    neg_auc_score = -1 * auc_score

    return {"loss": neg_auc_score, "status": STATUS_OK}

In [10]:
# search for model

X = train[all_features]
y = target


trials = Trials()

best_hyperparams = fmin(fn=xgb_search,
                       space=xgb_search_params,
                       algo=tpe.suggest,
                       max_evals=100,
                       trials=trials)

[11:14:00] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:15:42] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:18:20] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\s

[12:00:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:08:16] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:09:05] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\s

[12:37:49] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:40:00] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:45:53] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\s

[13:30:13] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:35:17] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:37:34] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\s

[14:36:46] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:44:07] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:47:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\s

[15:39:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:40:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480:                   
Parameters: { min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:44:01] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\s

In [11]:
best_hyperparams

{'colsample_bytree': 0.548359184501841,
 'learning_rate': 0.022343893412903975,
 'max_depth': 13.0,
 'min_child_weight': 8.355869997463596e-06,
 'min_data_in_leaf': 38.0,
 'min_split_gain': 0.8852413428906907,
 'num_leaves': 135.0,
 'reg_alpha': 0.6633147598728293,
 'reg_lambda': 37.66225460626474,
 'subsample': 0.7109750407827766,
 'subsample_for_bin': 190000.0}

In [14]:
xgbm_params = {
    'objective': 'binary:logistic',
    'metric': 'auc',
    'random_state': SEED,
    'n_estimators': 6000,
    'n_jobs': -1,
    
    
    'colsample_bytree': 0.548359184501841,
    'learning_rate': 0.007447964, # 0.022343893412903975,
    'max_depth': 13,
    'min_child_weight': 8.355869997463596e-06,
    'min_data_in_leaf': 38,
    'min_split_gain': 0.8852413428906907,
    'num_leaves': 135,
    'reg_alpha': 0.6633147598728293,
    'reg_lambda': 37.66225460626474,
    'subsample': 0.7109750407827766,
    'subsample_for_bin': 190000
}


In [15]:
#X = X.abs()
y = train["target"]
kf = KFold(n_splits=8, shuffle=True, random_state=1)
oof_2 = np.zeros(len(train))
score_list = []
fold = 1
test_preds_2 = []

# view accuracy
from sklearn.metrics import accuracy_score


for train_index, test_index in kf.split(train):
    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

    
    dtrain = X_train[all_features]
    dvalid = X_val[all_features]
    
    model = XGBClassifier(**xgbm_params) 

    evaluation = [(dvalid, y_val)]

    model.fit(dtrain, y_train,
              eval_set=evaluation, eval_metric='auc',
              early_stopping_rounds=100, verbose=False)

    val_preds = model.predict(dvalid)
    
    fpr, tpr, thresholds = roc_curve(y_val, val_preds, pos_label = 1)
    auc_score = auc(fpr, tpr)


    print(f"AUC Score Fold-{fold} : {auc_score}")

    oof_2[test_index] = val_preds

    
    score_list.append(auc_score)
    test_preds_2.append(model.predict_proba(test[all_features])[:, 1])
    fold+=1
    

np.mean(score_list)

[17:11:23] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { metric, min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


AUC Score Fold-1 : 0.7803789349245007
[17:30:21] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { metric, min_data_in_leaf, min_split_gain, num_leaves, subsample_for_bin } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


AUC Score Fold-2 : 0.7749281474061903
[17:50:00] WARNING: C:\Use

0.7757962721640623

In [16]:
print(np.mean(score_list))
score_list

0.7757962721640623


[0.7803789349245007,
 0.7749281474061903,
 0.7776873072464359,
 0.7733604853720927,
 0.7747591979829713,
 0.777251937508721,
 0.7748932205791517,
 0.7731109462924353]

In [17]:
train["2_preds"] = oof_2
test["2_preds"] = np.mean(test_preds_2,axis=0)

fpr, tpr, thresholds = roc_curve(train["target"], train["2_preds"], pos_label = 1)
auc_score = auc(fpr, tpr)
    
print(f"AUC Score Final : {auc_score}")

AUC Score Final : 0.7757897516690564


In [18]:
# save prediction
subm_df = test[["2_preds"]]
subm_df['id'] = test_id
subm_df = subm_df.rename(columns={"2_preds": "target"})
subm_df.to_csv("hyperopt_single_xgb_model.csv",index=False)

In [19]:
print('We are done. That is all, folks!')
finish_time = dt.datetime.now()
print("Finished at ", finish_time)
elapsed = finish_time - start_time
print("Elapsed time: ", elapsed)

We are done. That is all, folks!
Finished at  2021-03-23 20:26:29.600542
Elapsed time:  9:12:34.714090


# Historical Record Tracking

## XGBoost , without noisy features

- Final validation AUC locally - 0.7757897516690564
- public LB 0.88851

```
xgbm_params = {
    'objective': 'binary:logistic',
    'metric': 'auc',
    'random_state': SEED,
    'n_estimators': 6000,
    'n_jobs': -1,
    
    
    'colsample_bytree': 0.548359184501841,
    'learning_rate': 0.007447964, # 0.022343893412903975,
    'max_depth': 13,
    'min_child_weight': 8.355869997463596e-06,
    'min_data_in_leaf': 38,
    'min_split_gain': 0.8852413428906907,
    'num_leaves': 135,
    'reg_alpha': 0.6633147598728293,
    'reg_lambda': 37.66225460626474,
    'subsample': 0.7109750407827766,
    'subsample_for_bin': 190000
}

```

## Baseline lightgbm, with all features

- Final validation AUC locally - 0.7776592687653098
- public LB 0.882171

```
lgbm_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'random_state': SEED,
    'n_estimators': N_ESTIMATORS,
    'n_jobs': -1,
    'cat_feature': [x for x in range(len(cat_cols))],
    'bagging_seed': SEED,
    'feature_fraction_seed': SEED,
    
    
    'colsample_bytree': 0.8785258997000931,
    'learning_rate': 0.014051289,
    'max_depth': 11,
    'min_child_weight': 5.5803474120805336e-06,
    'min_data_in_leaf': 85,
    'min_split_gain': 0.23035992374930991,
    'num_leaves': 248,
    'reg_alpha': 0.5030438494317617,
    'reg_lambda': 27.412351627541167,
    'subsample': 0.5725588226322971,
    'subsample_for_bin': 160000
}
```

## Single lgbm without non-important cat features

- AUC Score Final (locally) : 0.777074298578011
- public lb 0.88821

cat cols dropped: ['cat5', 'cat7', 'cat8', 'cat10']

(jump from top 66% to top 47% in public lb as of Mar 6, 2021)

```
lgbm_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'random_state': SEED,
    'n_estimators': N_ESTIMATORS,
    'n_jobs': -1,
    'cat_feature': [x for x in range(len(cat_cols))],
    'bagging_seed': SEED,
    'feature_fraction_seed': SEED,
    
    
    'colsample_bytree': 0.7671990402625448,
     'learning_rate': 0.019767284,
     'max_depth': 15,
     'min_child_weight': 0.0002777396522195953,
     'min_data_in_leaf': 57,
     'min_split_gain': 0.21136452616481655,
     'num_leaves': 175,
     'reg_alpha': 0.11063887315135824,
     'reg_lambda': 18.001114258982668,
     'subsample': 0.9818248529067929,
     'subsample_for_bin': 180000
}


```
